In [ ]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import numpy as np
import copy
import time 
import matplotlib.pyplot as plt
import pickle # Lokales Speichern von Objekten
import keyboard
from scipy import interpolate

from GNM_Toolbox import *

dataset = load_dataset('Citeseer')

# Some Helping Functions

In [ ]:
# Gegeben sei eine target_list (a_0, a_1, a_2, ...)
# und eine out_list ((b_0, x), (b_1, x), (b_2, x), (b_3, x), ...)
# out_list darf tupel beliebiger Länge haben. 
# Gesucht wird eine Liste l von Indizes, sodass für i < len(target_list): abs(target_list[i] - out_list[l[i]][0]) minimal ist
def find_each_nearest(target_list, out_list, idx = 0):
    # Each list is expected to be sorted
    i, j = 0, 0
    result = list()
    while True:
        diff_0 = abs(target_list[i] - out_list[j][idx])
        diff_1 = abs(target_list[i] - out_list[j+1][idx])
        
        if diff_0 >= diff_1:
            j += 1
        elif diff_0 < diff_1:
            result.append(j)
            i += 1
        if i >= len(target_list):
            return result
        if j+1 >= len(out_list):
            while i < len(target_list):
                result.append(j)
                i += 1
            return result
            
def get_best_values_indices(targets, lambdas):
    lambdas.sort(key = lambda x: x[0])
    return find_each_nearest(targets, lambdas)

def create_mask_from_pi(data, pi):
    p = pi(data.x, data.y)
    mask = torch.tensor((np.random.binomial(size = p.shape[0], n = 1, p = p) == 1))        
    return mask.bool()

def split_known_mask_into_val_and_train_mask(known, ratio=0.8):
    val_mask = torch.zeros_like(known) == 1
    train_mask = torch.zeros_like(known) == 1
    for i in range(len(known)):
        if known[i] == True:
            if np.random.binomial(1, ratio) == 1:
                train_mask[i] = True
            else:
                val_mask[i] = True
    return val_mask, train_mask

def count_classes(y, mask):
    l = np.zeros((max(y)+1))
    for yy in y[mask]:
        l[yy] += 1
    return l

def calc_variance(y, mask):
    y_distribution = count_classes(y, mask)
    return np.var(y_distribution)
        
def insert_into_list(l, item, t):
    # l list, i item to insert, target
    def diff(a, b):
        return abs(a-b)
    N = len(l)
    if N == 0:
        l.insert(0, item)
        return
    d = diff(t, item[0])
    d_0 = diff(t, l[0][0])
    if d <= d_0:
        l.insert(0, item)
        return
    for i in range(N-1):
        d_0 = diff(t, l[i][0])
        d_1 = diff(t, l[i+1][0])
        if d_0 <= d and d <= d_1:
            l.insert(i+1, item)
            return
    l.append(item)

def disturb_y(y, probability): # y must have 2 classes
    y_dis = torch.zeros_like(y)
    for i in range(len(y)):
        if np.random.rand(1) < probability:
            y_dis[i] = 0 if np.random.rand(1) > 0.5 else 1
        else:
            y_dis[i] = y[i]
                
    return F.one_hot(y_dis, 2)

# root mean squared error
def RMSE(pi_est, pi_true):
    return torch.sqrt(F.mse_loss(pi_true, pi_est.view(pi_true.shape)))

# mean squared error
def MSE(pi_est, pi_true):
    return F.mse_loss(pi_true, pi_est)

# weighted mean squared error
def WMSE(pi_est, pi_true, weight):
    return torch.mean(weight * (pi_est - pi_true) ** 2)


# mean root error
def MRE(pi_est, pi_true):
    return torch.mean(torch.pow(abs(pi_est.view(pi_true.shape) - pi_true), 0.5))

# berechnet Gewichtungen für weighted mean square error
def calc_inverse_weights(pi_real):
    bins = torch.zeros(len(pi_real)).type(torch.float)
    pi = pi_real.detach().numpy()
    # Count 
    for p in pi_real:
        bins[int(p*100)] += 1
    bins = 1/bins
    bins[bins == (torch.tensor(1.)/0)] = 999999999
    return torch.sqrt(bins)

def eval_pi(pi_true, pi_est, diff=0.01):
    pi_diff = abs(pi_true.view(pi_est.shape)-pi_est)
    pi_binar = [(p < diff) for p in pi_diff]
    return sum(pi_binar)/(1. * pi_true.shape[0])

def var(l, mean):
    # l: list of floats, 
    # mean: float
    return np.mean((l-mean)**2)

def plot_data(all_models, colors):
    names = list(all_models.keys())
    M = len(names)
    
    plt.figure(None, (18, 8))
    
    for name, color in zip(names, colors):
        # print point cloud
        data = np.array(all_models[name]).T
        plt.plot(*data, 'x', alpha = 1, color=color)
        
        # print mean function
        lambdas = data[0]
        values = data[1]
        mini = np.min(lambdas)
        maxi = np.max(lambdas)
        width = (maxi-mini)/5
        xn = np.linspace(mini, maxi, 30)
        yn_mean = np.zeros_like(xn)
        yn_lower_var = np.zeros_like(xn)
        yn_upper_var = np.zeros_like(xn)
        for i in range(len(xn)):
            ys = list()
            # Find all values in given interval [xn[i] +- width/2]
            for j in range(len(values)):
                if lambdas[j] <= xn[i] + width/2 and lambdas[j] >= xn[i] - width/2:
                    ys.append(values[j])
            ys = np.sort(ys)
            mean = np.mean(ys)
            ys_lower = [y for y in ys if y < mean]
            ys_upper = [y for y in ys if y > mean]
            
            yn_mean[i] = mean
            yn_lower_var[i] = np.sqrt(var(ys_lower, mean))
            yn_upper_var[i] = np.sqrt(var(ys_upper, mean))
            
            
        mean_spline = interpolate.UnivariateSpline(xn, yn_mean)
        var_upper_spline = interpolate.UnivariateSpline(xn, yn_mean + yn_upper_var)
        var_lower_spline = interpolate.UnivariateSpline(xn, yn_mean - yn_lower_var)
        mean_spline.set_smoothing_factor(0.0001)
        var_upper_spline.set_smoothing_factor(0.001)
        var_lower_spline.set_smoothing_factor(0.001)
        
        #plt.plot(xn, yn_mean, color=color, label=name)
        plt.plot(xn, mean_spline(xn), color=color, label=name)
        
        plt.plot(xn, var_lower_spline(xn), alpha=0.3, color=color)
        plt.plot(xn, var_upper_spline(xn), alpha=0.3, color=color)
        plt.fill_between(xn, var_lower_spline(xn), var_upper_spline(xn), alpha=0.04, color=color)

    plt.legend(fontsize=15)
    plt.xlabel(r'$\lambda$', fontsize=20)
    plt.ylabel(r'Genauigkeit', fontsize=20)

# Data Setup

In [ ]:
def h(x):
    a0, a1, a2, a3 = 13, 4, 15, 15
    return torch.exp(torch.sum(x, dim=1)/a0 - a1) - ((torch.sum(x, dim=1) - a2 ) / a3)
    
def pi(X, y):
    y0 = F.one_hot(y, 6)
    # y should be one-hot encoded
    a0, a1, a2 = -torch.log(torch.tensor(20.)), 1, torch.tensor([[5, 0.5, 0.2, 1, 0.5, 2]]).view((6, 1))
    return torch.sigmoid(a0 + a1 * h(X).view((-1, 1)) + y0.type(torch.float) @ a2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)
data.num_classes = dataset.num_classes

# Create Masks

In [ ]:
# Finde Masken mit passenden Lambdas
# Erstelle dazu 2000 Masken und wähle dann die Masken aus, dessen Lambdas den targets entsprechen
masks0 = list()
for i in range(200):
    mask = create_mask_from_pi(data, pi)
    val_mask, train_mask = split_known_mask_into_val_and_train_mask(mask)
    l = calc_variance(data.y, train_mask)
    masks0.append([train_mask, val_mask, l])
    print(i, end='\r')
masks0.sort(key=lambda x: x[2])
targets = np.linspace(10, 120, 200) # choose masks depending on lambdas
idxs = find_each_nearest(targets, masks0, idx=2)
idxs = list(dict.fromkeys(idxs)) # remove duplicates
masks = [masks0[i] for i in idxs]
print('Got {} masks'.format(len(masks)))
#np.random.shuffle(masks)

# Evaluate Models

In [ ]:
choosen_masks = masks
IT_PER_MASK = 2
M = len(choosen_masks)
sm_models = list()
gnm_models = list()
gnma_models = list()

In [ ]:
# Iteriere über Masken
t_0 = time.time()
for j, mask_tupel in enumerate(choosen_masks):
    train_mask, val_mask, l = mask_tupel

    # Trainiere jeweils N Modelle
    for k in range(IT_PER_MASK):
        print_status(j * IT_PER_MASK + k, M * IT_PER_MASK, t_0)
    
        # In gnm.py wird beim Sampling die Methode np.random.multinomial.
        # Aufgrund von der Ungenauigkeit von Floats/Doubles kommt es hier in sehr seltenen Fälle vor, dass
        # die Summe über die Wahrscheinlichkeiten 1 überschreitet und somit ein Fehler geworfen wird.
        # Damit das den durchaus langen Prozess des Trainings nicht abbricht, wird dieser hier ggf abgefangen.
        try:
            _, _, acc_sm, _ = train_one_net(data, train_mask, val_mask)
            sm_models.append([l, acc_sm])
        except Exception as e: 
            print('SM: {}'.format(e))
        
        try:
            _, _, acc_gnm = train_net_with_gnm(data, train_mask, val_mask)
            gnm_models.append([l, acc_gnm[-1]])
        except Exception as e: 
            print('GNM: {}'.format(e))
            
        try:
            _, _, acc_gnma = train_net_with_gnm_adapted(data, train_mask, val_mask)
            gnma_models.append([l, acc_gnma])
        except Exception as e: 
            print('GNMn: {}'.format(e))
        
        
        
        all_models = {'GNM': gnm_models, 'GNMn': gnma_models, 'SM': sm_models}
#pickle_write('algorithm-analysis-citeseer-2.pkl', all_models)

In [ ]:
print(all_models)
plot_data(all_models, ['red', 'blue', 'green']) # Hierfür müssen ausreichend Daten vorhanden sein.